In [1]:
import pandas as pd
import seaborn as sns
import altair as alt


asec = pd.read_pickle("ipums/asec.pkl")

# Filtering
#asec = asec[asec["WORKLY"] == 2] # worked last year
asec = asec[asec["FULLPART"] == 1] # fulltime
asec = asec[asec["UHRSWORKLY"] >= 35] # work at least 35 hours

# Adjust inflation to 2022 dollars
asec["INCWAGE_U"] = asec["INCWAGE"] * asec["CPI99"] * 1.797

# Calculate hourly wage based on adjusted wage
asec["HOURLYWAGE"] = asec["INCWAGE_U"] / (asec["WKSWORK1"] * asec["UHRSWORKLY"])


asec["SEX"] = asec["SEX"].astype(str).replace({"1": "Male", "2": "Female"})


In [2]:
asec.keys()

Index(['YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'ASECFLAG', 'HFLAG',
       'ASECWTH', 'CPI99', 'STATEFIP', 'PERNUM', 'WTFINL', 'CPSIDV', 'CPSIDP',
       'ASECWT', 'AGE', 'SEX', 'RACE', 'MARST', 'EDUC', 'EDDIPGED', 'OCCLY',
       'INDLY', 'CLASSWLY', 'WORKLY', 'WKSWORK1', 'UHRSWORKLY', 'FULLPART',
       'INCWAGE', 'DISABWRK', 'VETLAST', 'INCWAGE_U', 'HOURLYWAGE'],
      dtype='object')

In [22]:
asec

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,HFLAG,ASECWTH,CPI99,STATEFIP,...,CLASSWLY,WORKLY,WKSWORK1,UHRSWORKLY,FULLPART,INCWAGE,DISABWRK,VETLAST,INCWAGE_U,HOURLYWAGE
0,1976,1,3,NaN,19760304072900,1,<NA>,1729.13,3.097,9,...,22,<NA>,47,40,1,11000,<NA>,1,61218.399,32.562978
1,1976,1,3,NaN,19760304072900,1,<NA>,1729.13,3.097,9,...,22,<NA>,52,40,1,8000,<NA>,0,44522.472,21.405035
2,1976,2,3,NaN,19760304073000,1,<NA>,1516.90,3.097,9,...,22,<NA>,52,40,1,11600,<NA>,9,64557.5844,31.0373
3,1976,2,3,NaN,19760304073000,1,<NA>,1516.90,3.097,9,...,22,<NA>,52,40,1,6609,<NA>,0,36781.127181,17.683234
6,1976,3,3,NaN,19760304073100,1,<NA>,1282.68,3.097,9,...,22,<NA>,52,40,1,8608,<NA>,0,47906.179872,23.031817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7051607,2023,88976,3,NaN,0,1,<NA>,445.78,0.569,15,...,27,2,52,40,1,60000,1,0,61349.58,29.49499
7051608,2023,88977,3,NaN,0,1,<NA>,413.20,0.569,15,...,22,2,52,40,1,50000,1,0,51124.65,24.579159
7051609,2023,88977,3,NaN,0,1,<NA>,413.20,0.569,15,...,27,2,52,40,1,40000,1,0,40899.72,19.663327
7051612,2023,88978,3,NaN,0,1,<NA>,416.37,0.569,15,...,27,2,52,40,1,33000,1,0,33742.269,16.222245


In [24]:
# Gender median hourly wage
median_wages = asec.groupby(["YEAR", "SEX"])["HOURLYWAGE"].median().reset_index()
print(median_wages)
alt.Chart(median_wages).mark_line(point=True).encode(
    x="YEAR:O",
    y="HOURLYWAGE:Q",
    color="SEX:N",
)

    YEAR     SEX  HOURLYWAGE
0   1976  Female   17.124028
1   1976    Male   27.180317
2   1977  Female   17.339416
3   1977    Male   26.893828
4   1978  Female   17.289836
..   ...     ...         ...
91  2021    Male     28.9317
92  2022  Female   24.653458
93  2022    Male   27.628875
94  2023  Female   24.382525
95  2023    Male   27.357672

[96 rows x 3 columns]


alt.Chart(...)

In [38]:
gender_wages = (
    asec.groupby(["YEAR", "SEX"])["HOURLYWAGE"]
    .apply(lambda x: x.quantile([0.15, 0.5, 0.9]))
    .unstack()
    .reset_index()
)
gender_wages.columns = ["YEAR", "SEX", "15th", "50th", "90th"]
wage_melted = gender_wages.melt(id_vars=['YEAR', 'SEX'], value_vars=['15th', '50th', '90th'],
                                    var_name='Percentile', value_name='HOURLYWAGE')


def create_percentile_chart(percentile_name, ymin=0, ymax=80):
    # Filter data for the specified percentile
    subset = wage_melted[wage_melted['Percentile'] == percentile_name]
    chart = alt.Chart(subset).mark_line(point=True).encode(
        x=alt.X('YEAR:O', axis=alt.Axis(values=list(range(1980, 2030, 10))), title='Year'),
        y=alt.Y('HOURLYWAGE:Q', scale=alt.Scale(domain=[ymin, ymax]), title='Hourly Wage'),
        color=alt.Color('SEX:N', legend=alt.Legend(title="Sex"),
                        scale=alt.Scale(domain=['Male', 'Female'])),  # Custom colors
        shape=alt.Shape('SEX:N', legend=alt.Legend(title="Sex"),
                        scale=alt.Scale(domain=['Male', 'Female'], range=['square', 'triangle-up'])),  # Custom shapes
        tooltip=['YEAR', 'SEX', 'HOURLYWAGE']
    ).properties(
        width=400,
        title=f'Hourly Wage {percentile_name} by Sex Over Years'
    )
    return chart

create_percentile_chart("15th", 8, 15) | create_percentile_chart("50th", 15, 30) | create_percentile_chart("90th", 20, 80)

alt.HConcatChart(...)

In [47]:
print(wage_melted)

     YEAR                  SEX Percentile HOURLYWAGE_PERCENT
0    1976  Female as % of Male       15th           0.861111
1    1977  Female as % of Male       15th           0.945455
2    1978  Female as % of Male       15th           0.923077
3    1979  Female as % of Male       15th           0.941325
4    1980  Female as % of Male       15th           1.033365
..    ...                  ...        ...                ...
139  2019  Female as % of Male       90th              0.775
140  2020  Female as % of Male       90th                0.8
141  2021  Female as % of Male       90th           0.776923
142  2022  Female as % of Male       90th           0.792793
143  2023  Female as % of Male       90th           0.788061

[144 rows x 4 columns]


In [62]:
gender_wages = (
    asec.groupby(["YEAR", "SEX"])["HOURLYWAGE"]
    .apply(lambda x: x.quantile([0.15, 0.5, 0.9]))
    .unstack()
    .reset_index()
)
gender_wages.columns = ["YEAR", "SEX", "15th", "50th", "90th"]


# Separate data into male and female
male_wages = gender_wages[gender_wages['SEX'] == 'Male'].drop('SEX', axis=1).set_index('YEAR')
female_wages = gender_wages[gender_wages['SEX'] == 'Female'].drop('SEX', axis=1).set_index('YEAR')

# Calculate female wage as a percentage of male wage
wage_percentages = female_wages.divide(male_wages).reset_index()
wage_percentages['SEX'] = 'Female as % of Male'

# Melting the DataFrame for easier plotting with Altair
wage_melted = wage_percentages.melt(id_vars=['YEAR', 'SEX'], value_vars=['15th', '50th', '90th'],
                                    var_name='Percentile', value_name='HOURLYWAGE_PERCENT')
wage_melted["HOURLYWAGE_PERCENT"] *= 100

def create_percentile_chart(percentile_name, ymin=50, ymax=110):
    # Filter data for the specified percentile
    subset = wage_melted[wage_melted['Percentile'] == percentile_name]
    chart = alt.Chart(subset).mark_line(point=True).encode(
        x=alt.X('YEAR:O', axis=alt.Axis(values=list(range(1980, 2030, 10))), title='Year'),
        y=alt.Y('HOURLYWAGE_PERCENT:Q', scale=alt.Scale(domain=[ymin, ymax]), title='Pay Gap (%)'),
        tooltip=['YEAR', 'SEX', 'HOURLYWAGE_PERCENT']
    ).properties(
        width=400,
        title=f'Pay gap at {percentile_name} of women as a % of the {percentile_name} hourly wage of men'
    )
    return chart

create_percentile_chart("15th") | create_percentile_chart("50th") | create_percentile_chart("90th")

alt.HConcatChart(...)